<a href="https://colab.research.google.com/github/ayaelassal/ayaelassal/blob/main/ComfyUI_Installer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# @title خلية التحقق من وجود GPU
!nvidia-smi

Thu Oct 23 09:21:16 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:

# @title الخلية 1: التحقق من المفاتيح السرية
import os
from google.colab import userdata

print("--- التحقق من المفاتيح السرية ---")
# التحقق من وجود المفاتيح المطلوبة في Colab Secrets
try:
    hf_token = userdata.get('HF_TOKEN')
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    if not hf_token or not ngrok_token:
        raise ValueError("أحد المفاتيح (HF_TOKEN أو NGROK_AUTH_TOKEN) غير موجود.")
    print("✅ تم العثور على جميع المفاتيح السرية المطلوبة.")
except Exception as e:
    print(f"❌ خطأ: {e}")
    print("   الرجاء التأكد من إضافة HF_TOKEN و NGROK_AUTH_TOKEN في قسم الأسرار (🔑).")

--- التحقق من المفاتيح السرية ---
❌ خطأ: Secret HF_TOKEN does not exist.
   الرجاء التأكد من إضافة HF_TOKEN و NGROK_AUTH_TOKEN في قسم الأسرار (🔑).


In [5]:

# @title الخلية 2: التثبيت الشامل والنهائي (الإصدار الكامل)
import os
import urllib.parse
import subprocess
from huggingface_hub import hf_hub_download
from google.colab import userdata

# ==================================================================================
# ==> قسم الإعدادات: أضف روابطك هنا <==
# ==================================================================================

#@markdown # ⚙️ **قسم الإعدادات (أضف روابطك هنا)**
#@markdown ---
#@markdown ### 📥 **نماذج إضافية (Checkpoints, LoRAs, VAEs)**
#@markdown الصق روابط التحميل المباشرة هنا. سيتم تجاهل أي سطر لا يبدأ بـ "http".
ADDITIONAL_MODELS = [
    # تم تحديث هذا الرابط بالنموذج الصحيح المطلوب
    "https://huggingface.co/Comfy-Org/stable-diffusion-v1-5-archive/resolve/main/v1-5-pruned-emaonly-fp16.safetensors?download=true",
    "ضع رابط النموذج الثاني هنا",
    "ضع رابط النموذج الثالث هنا",
    "ضع رابط النموذج الرابع هنا",
    "ضع رابط النموذج الخامس هنا",
]

#@markdown ---
#@markdown ### ➕ **إضافات / Custom Nodes (من GitHub)**
#@markdown الصق روابط `git` هنا (يجب أن تنتهي بـ `.git`). سيتم تجاهل أي سطر لا يبدأ بـ "http".
ADDITIONAL_CUSTOM_NODES = [
    "ضع رابط الإضافة الأولى هنا",
    "ضع رابط الإضافة الثانية هنا",
    "ضع رابط الإضافة الثالثة هنا",
    "ضع رابط الإضافة الرابعة هنا",
    "ضع رابط الإضافة الخامسة هنا",
]


# ==================================================================================
# ==> يبدأ الكود الرئيسي من هنا (لا تقم بالتعديل) <==
# ==================================================================================

# --- 1/6: تثبيت ComfyUI بشكل كامل ---
print("--- 1/6: جاري تحميل وتثبيت نسخة كاملة من ComfyUI...")
COMFYUI_DIR = "/content/ComfyUI"
if not os.path.exists(COMFYUI_DIR):
    get_ipython().system(f'git clone --recurse-submodules https://github.com/comfyanonymous/ComfyUI.git "{COMFYUI_DIR}"')
else:
    print("   - ✅ ComfyUI موجود بالفعل.")
print("✅ تم تثبيت ComfyUI بشكل كامل وسليم.")

# --- 2/6: تثبيت المتطلبات البرمجية الأساسية ---
print("\n--- 2/6: تثبيت كل المتطلبات البرمجية...")
%cd $COMFYUI_DIR
print("   - إزالة xformers من المتطلبات لتجنب التعارض...")
get_ipython().system(f"sed -i '/xformers/d' requirements.txt")
print("   - تثبيت المتطلبات الأساسية...")
get_ipython().system('pip install -r requirements.txt --quiet')
print("   - تثبيت مكتبة uv الإضافية...")
get_ipython().system('pip install uv --quiet')
print("✅ تم تثبيت كل المتطلبات بنجاح.")


# --- 3/6: تثبيت حزمة المبدع القوية (Power User Custom Nodes) ---
print("\n--- 3/6: تثبيت حزمة الإضافات للمحترفين...")
nodes_dir = os.path.join(COMFYUI_DIR, "custom_nodes")
%cd $nodes_dir

power_user_nodes = {
    "ComfyUI-Manager": "https://github.com/ltdrdata/ComfyUI-Manager.git",
    "ComfyUI-Impact-Pack": "https://github.com/ltdrdata/ComfyUI-Impact-Pack.git",
    "was-node-suite-comfyui": "https://github.com/WASasquatch/was-node-suite-comfyui.git",
    "ComfyUI-ControlNetAux": "https://github.com/Fannovel16/ComfyUI-ControlNetAux.git",
    "ComfyUI-AnimateDiff-Evolved": "https://github.com/Kosinkadink/ComfyUI-AnimateDiff-Evolved.git",
    "ComfyUI_IPAdapter_plus": "https://github.com/cubiq/ComfyUI_IPAdapter_plus.git",
    "efficiency-nodes-comfyui": "https://github.com/jags111/efficiency-nodes-comfyui.git",
    "ComfyUI-Advanced-ControlNet": "https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet.git",
    "ComfyUI-VideoHelperSuite": "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git",
    "Derfuu_ComfyUI_ModdedNodes": "https://github.com/Derfuu/Derfuu_ComfyUI_ModdedNodes.git",
}
os.environ['GIT_TERMINAL_PROMPT'] = '0'

for name, url in power_user_nodes.items():
    if not os.path.exists(name):
        print(f"   - جاري تثبيت: {name}")
        subprocess.run(['git', 'clone', url, name], check=False, capture_output=True, text=True)
    else:
        print(f"   - ✅ {name} موجود بالفعل.")

print("\n   - تثبيت متطلبات الإضافات...")
for name in power_user_nodes:
    req_file = os.path.join(nodes_dir, name, "requirements.txt")
    if os.path.exists(req_file):
        get_ipython().system(f'pip install -r {req_file} --quiet')
print("✅ تم تثبيت حزمة الإضافات الأساسية.")


# --- 4/6: تثبيت الإضافات والنماذج المخصصة من إعداداتك ---
print("\n--- 4/6: تثبيت الإضافات والنماذج المخصصة التي أضفتها...")
checkpoints_dir = os.path.join(COMFYUI_DIR, "models", "checkpoints")

# تثبيت الإضافات المخصصة
custom_nodes_to_install = [url for url in ADDITIONAL_CUSTOM_NODES if url.strip().startswith("http")]
if custom_nodes_to_install:
    print("   - جاري تثبيت الإضافات المخصصة...")
    for url in custom_nodes_to_install:
        try:
            repo_name = url.split('/')[-1].replace('.git', '')
            print(f"     - Cloning: {repo_name}")
            subprocess.run(['git', 'clone', url, repo_name], check=True, capture_output=True, text=True)
        except Exception as e:
            print(f"     - ❌ فشل تثبيت: {url}. الخطأ: {e.stderr if hasattr(e, 'stderr') else e}")
else:
    print("   - لا توجد إضافات مخصصة لتثبيتها.")

# تحميل النماذج المخصصة
models_to_download = [url for url in ADDITIONAL_MODELS if url.strip().startswith("http")]
if models_to_download:
    print("   - جاري تحميل النماذج المخصصة...")
    for url in models_to_download:
        try:
            # Handle special Hugging Face URLs with ?download=true
            if "huggingface.co" in url and "?download=true" in url:
                # Use a different way to get filename for these URLs
                repo_id = url.split('/')[3] + '/' + url.split('/')[4]
                # A bit of guesswork here for the actual filename, might need adjustment
                filename = url.split('/')[-1].split('?')[0]
            else:
                parsed_url = urllib.parse.urlparse(url)
                filename = os.path.basename(parsed_url.path)

            destination_path = os.path.join(checkpoints_dir, filename)
            if not os.path.exists(destination_path):
                print(f"     - Downloading: {filename}")
                get_ipython().system(f'wget -O "{destination_path}" "{url}"')
            else:
                print(f"     - ✅ الملف {filename} موجود بالفعل. سيتم التخطي.")
        except Exception as e:
            print(f"     - ❌ فشل تحميل: {url}. الخطأ: {e}")
else:
    print("   - لا توجد نماذج مخصصة لتحميلها.")
print("✅ اكتمل تثبيت الإضافات المخصصة.")


# --- 5/6: تحميل النموذج الأساسي (SD3) ---
print("\n--- 5/6: تحميل النموذج الأساسي...")
hf_token = userdata.get('HF_TOKEN')
model_filename = "sd3_medium_incl_clips.safetensors"
final_model_path = os.path.join(checkpoints_dir, model_filename)

if not (os.path.exists(final_model_path) and os.path.getsize(final_model_path) > 5_000_000_000):
    print(f"   - 📂 نموذج SD3 غير موجود أو غير مكتمل، جاري التحميل...")
    try:
        hf_hub_download(
            repo_id="stabilityai/stable-diffusion-3-medium",
            filename=model_filename,
            local_dir=checkpoints_dir,
            local_dir_use_symlinks=False,
            token=hf_token
        )
        print("   - ✅ تم تحميل نموذج SD3 بنجاح.")
    except Exception as e:
        print(f"   - ❌ فشل تحميل نموذج SD3. الخطأ: {e}")
else:
    print("   - ✅ نموذج SD3 موجود بالفعل ومكتمل.")


# --- 6/6: الخطوة النهائية ---
print("\n🎉🎉🎉 اكتمل الإعداد الكامل والنهائي! 🎉🎉🎉")

--- 1/6: جاري تحميل وتثبيت نسخة كاملة من ComfyUI...
Cloning into '/content/ComfyUI'...
remote: Enumerating objects: 26117, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 26117 (delta 28), reused 5 (delta 5), pack-reused 26071 (from 3)
Receiving objects: 100% (26117/26117), 73.97 MiB | 26.00 MiB/s, done.
Resolving deltas: 100% (17602/17602), done.
✅ تم تثبيت ComfyUI بشكل كامل وسليم.

--- 2/6: تثبيت كل المتطلبات البرمجية...
/content/ComfyUI
   - إزالة xformers من المتطلبات لتجنب التعارض...
   - تثبيت المتطلبات الأساسية...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.4/86.4 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 134.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━

SecretNotFoundError: Secret HF_TOKEN does not exist.

In [7]:

# @title الخلية 3: المشغّل السريع (بدون Drive)
import os
import time
import re
from urllib import request, error
from google.colab import userdata

def wait_for_service(port, timeout=180): # 3 دقائق كافية الآن
    print(f"⏳ جاري انتظار الخادم ليكون جاهزًا...", end="")
    start_time = time.time()
    while time.time() - start_time < timeout:
        try:
            with request.urlopen(f"http://localhost:{port}/", timeout=1):
                print("\n✅ الخادم يعمل الآن وجاهز للاستقبال.")
                return True
        except (error.URLError, ConnectionRefusedError):
            print(".", end="", flush=True)
            time.sleep(2)
    print("\n❌ فشل تشغيل الخادم في الوقت المحدد.")
    return False

PORT = 8188
COMFYUI_DIR = "/content/ComfyUI"
LOG_FILE = "/content/comfyui_run.log"

print("🧹 خطوة التنظيف: التأكد من أن البورت فارغ...")
get_ipython().system(f'fuser -k {PORT}/tcp')
time.sleep(2)
print("✅ البورت الآن جاهز للاستخدام.")

print(f"🚀 جاري تشغيل ComfyUI في الخلفية...")
%cd $COMFYUI_DIR
get_ipython().system_raw(f'python main.py --listen 0.0.0.0 --port {PORT} --preview-method auto > {LOG_FILE} 2>&1 &')

if wait_for_service(PORT):
    print("\n🔗 جاري محاولة إنشاء رابط عام عبر Cloudflared...")
    if not os.path.exists("/content/cloudflared"):
        get_ipython().system('wget -q -O /content/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64')
        get_ipython().system('chmod +x /content/cloudflared')
    get_ipython().system_raw(f'/content/cloudflared tunnel --url http://localhost:{PORT} --no-autoupdate > /content/cloudflared.log 2>&1 &')
    time.sleep(5)
    link = None
    try:
        with open("/content/cloudflared.log", "r") as f:
            content = f.read()
            match = re.search(r"https://[A-Za-z0-9_-]+\.trycloudflare\.com", content)
            if match:
                link = match.group(0)
    except FileNotFoundError: pass

    if link:
        print("\n" + "="*70)
        print("🎉🎉🎉 نجاح! هذا هو رابط ComfyUI الخاص بك. اضغط عليه لفتحه. 🎉🎉🎉")
        print(link)
        print("="*70)
    else: # Fallback to ngrok
        print("\n⚠️ فشل Cloudflared، جاري محاولة استخدام ngrok...")
        from pyngrok import ngrok
        ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
        ngrok.set_auth_token(ngrok_token)
        public_url = ngrok.connect(PORT)
        print("\n" + "="*70)
        print("🎉🎉🎉 نجاح! هذا هو رابط ComfyUI الخاص بك عبر ngrok. 🎉🎉🎉")
        print(public_url)
        print("="*70)
else:
    print("\n" + "="*70)
    print("‼️ فشل الخادم في البدء. جاري عرض سجل الأخطاء:")
    print("="*70)
    get_ipython().system(f'cat {LOG_FILE}')

🧹 خطوة التنظيف: التأكد من أن البورت فارغ...
8188/tcp:             3712
✅ البورت الآن جاهز للاستخدام.
🚀 جاري تشغيل ComfyUI في الخلفية...
/content/ComfyUI
⏳ جاري انتظار الخادم ليكون جاهزًا............
✅ الخادم يعمل الآن وجاهز للاستقبال.

🔗 جاري محاولة إنشاء رابط عام عبر Cloudflared...

🎉🎉🎉 نجاح! هذا هو رابط ComfyUI الخاص بك. اضغط عليه لفتحه. 🎉🎉🎉
https://tulsa-wma-recorders-bool.trycloudflare.com


In [ ]:
# 🟢 رفع النموذج من جهازك إلى Google Colab
from google.colab import files
uploaded = files.upload()


In [9]:
# 🟢 تحميل نموذج RealVisXL تلقائيًا من رابط مباشر
import os
model_url = "https://huggingface.co/RealVisXL/realvisxl-v50/resolve/main/realvisxlV50_v50LightningBakedvae.safetensors"
model_path = "/content/ComfyUI/models/checkpoints/realvisxlV50_v50LightningBakedvae.safetensors"

# إنشاء المجلد إذا لم يكن موجود
os.makedirs(os.path.dirname(model_path), exist_ok=True)

# تحميل النموذج
!wget -O "{model_path}" "{model_url}"

print("✅ تم تحميل النموذج RealVisXL بنجاح!")


--2025-10-23 09:45:05--  https://huggingface.co/RealVisXL/realvisxl-v50/resolve/main/realvisxlV50_v50LightningBakedvae.safetensors
Resolving huggingface.co (huggingface.co)... 3.165.160.61, 3.165.160.12, 3.165.160.11, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.61|:443... connected.
HTTP request sent, awaiting response... 401 Unauthorized

Username/Password Authentication Failed.
✅ تم تحميل النموذج RealVisXL بنجاح!
